<h1>Equal Weight S&P 500 Index Fund</h1>

<h2>Introduction</h2>

In this project we create a Python script that accepts the value of a portfolio and outputs how many shares of each S&P 500 company one should purchase to get an equal-weight version of the fund.

<h2>Library Imports</h2>

In [9]:
import numpy as np
import pandas as pd
import requests as req
import xlsxwriter
import math

<h2>Stock Imports</h2>

We import a list of stocks but it is not dynamic. It is from the excel file "constituents.csv".



In [10]:
stocks = pd.read_csv("constituents.csv")
stocks

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",3/4/1957,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",7/26/2017,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",3/4/1957,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",10/6/1997,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",12/23/2019,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",8/7/2001,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",6/22/2001,109380,1873


<h2>Collecting the data we need</h2>

For this project we need the marketCap and price of each of the stocks. We will collect the data using the yfinance module and store it in a dataframe.

First we import the yfinance module

In [11]:
import yfinance as yf

Then we prepare our data frame

In [12]:
myColumns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares"]
final_frame = pd.DataFrame(columns = myColumns)

Now we loop through the symbols in the stocks file and make the calls to collect the data with yfinance

In [13]:
for stock in stocks["Symbol"]:
    #print(stock)
    data = yf.Ticker(stock).info
    
    marketCap = data['marketCap']
    price = data['currentPrice']
    
    final_frame = final_frame.append(
    pd.Series(
        [
            stock,
            price,
            marketCap,
            'N/A' #We use N/A here because we haven't calculated it yet
        ],
        index = myColumns #This line tells the append method which colums to put the data under
        
    ), ignore_index = True #We always need to add this to the append method
    )

C:\Users\USER\AppData\Local\Temp\ipykernel_31500\1301388244.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_frame = final_frame.append(
C:\Users\USER\AppData\Local\Temp\ipykernel_31500\1301388244.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_frame = final_frame.append(
C:\Users\USER\AppData\Local\Temp\ipykernel_31500\1301388244.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_frame = final_frame.append(
C:\Users\USER\AppData\Local\Temp\ipykernel_31500\1301388244.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_frame = final_frame.append(
C:\Users\USER\AppData\Local\Temp\ipykernel_31500\1301388244.

<h2>Calculating the number of shares to buy</h2>

First we take the size of the portfolio as the input

In [14]:
while(True):
    portfolio_size = input("Enter the value of your portfolio: ")
    try:
        val = float(portfolio_size)
        print(val)
        break
    except ValueError:
        print("Did you input a number?\n Please input a number.")

100000000.0


Now for the calculations.

In [15]:
position_size = val/len(final_frame.index)
for i in range(len(final_frame.index)):
    final_frame.loc[i, "Number of Shares"] = math.floor(position_size/final_frame.loc[i, "Stock Price"])
final_frame

,Ticker,Stock Price,Market Capitalization,Number of Shares
0,MMM,99.955,55142375424,1988
1,AOS,69.630,10475276288,2855
2,ABT,102.060,177477238784,1947
3,ABBV,136.140,240190439424,1460
4,ACN,309.045,195328802816,643
...,...,...,...,...
498,YUM,131.905,36944875520,1507
499,ZBRA,278.680,14332595200,713
500,ZBH,130.770,27435284480,1520
501,ZION,30.730,4551143424,6469


<h2>Saving the output in Excel</h2>

First we initialize the Xlsxwriter object.

In [17]:
writer = pd.ExcelWriter("recommended_trades.xlsx", engine = 'xlsxwriter')
final_frame.to_excel(writer, "Recommended Trades", index = False)

Now we create the formats we want to use

In [18]:
background_color = "#191970"
font_color = "#ffffff"
string_format = writer.book.add_format(
    {
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1

    }
)
dollar_format = writer.book.add_format(
    {
        "num_format": "$0.00",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)
integer_format = writer.book.add_format(
    {
        "num_format": "0",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

Now we'll apply the formattings (I know this part seems like pure memorization; it is. Just go with it, and learn how to use it. )

In [19]:
column_formats = {
    "A": ["Ticker", string_format],
    "B": ["Stock Price", dollar_format],
    "C": ["Market Capitalization", dollar_format],
    "D": ["Number of Shares", integer_format]
    
}

for column in column_formats.keys():
    writer.sheets["Recommended Trades"].set_column(f"{column}:{column}", 20, column_formats[column][1])
    writer.sheets["Recommended Trades"].write(f"{column}1", column_formats[column][0], column_formats[column][1])

Finally we save the output

In [20]:
writer.save()

C:\Users\USER\AppData\Local\Temp\ipykernel_31500\934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
